In [ ]:
!pip install feedparser
import feedparser

rss_url = "https://rss.nytimes.com/services/xml/rss/nyt/Technology.xml"
feed = feedparser.parse(rss_url)

for entry in feed.entries[:5]:
    print(f"Title: {entry.title}")
    print(f"Link: {entry.link}\n")

# 🛠️ JIRA API Demo – Quick Start Guide (No .env file)


## ✅ Step 1: Create a Free Atlassian JIRA Account

1. Go to: [https://www.atlassian.com/software/jira/free](https://www.atlassian.com/software/jira/free)
2. Sign up with your email.
3. After setup, your JIRA URL will look like:
https://<your-subdomain>.atlassian.net

---

## ✅ Step 2: Create a JIRA Project

1. Click on **Projects** in the top navigation.
2. Click **Create project**.
3. Choose a template like **Scrum**, **Kanban**, or **Team-managed software**.
4. Set a name (e.g. `Demo Project`), and confirm the project key (e.g. `DEMO`).

---

## ✅ Step 3: Generate an API Token

1. Visit: [https://id.atlassian.com/manage-profile/security](https://id.atlassian.com/manage-profile/security)
2. Scroll down to **API token** → Click **Create and manage API tokens**
3. Click **Create API token**
4. Name it something like `"JIRA Demo Client"`
5. Click **Create** and **copy** the token (you won’t see it again)


In [ ]:
!pip install jira

In [ ]:
from jira import JIRA

# 🔐 Replace these values with your real credentials
JIRA_SUBDOMAIN = "<your_jira_subdomain>"  # e.g., "mycompany"
JIRA_EMAIL = "<your_email>"              # e.g., "you@example.com"
JIRA_API_TOKEN = "<your_jira_api_token>" # Create via https://id.atlassian.com/manage/api-tokens
JIRA_PROJECT_KEY = "<your_project_key>"  # e.g., "SCRUM"


# Setup the client
jira = JIRA(
    server=f"https://{JIRA_SUBDOMAIN}.atlassian.net",
    basic_auth=(JIRA_EMAIL, JIRA_API_TOKEN)
)

# Fetch and display issues
issues = jira.search_issues(f'project={JIRA_PROJECT_KEY}', maxResults=5)
for issue in issues:
    print(f"{issue.key}: {issue.fields.summary}")


# 🛠️ Slack API Demo – Quick Start Guide (No .env file)

## ✅ Step 1: Create a Free Slack Workspace

1. Go to: [https://slack.com/get-started#/create](https://slack.com/get-started#/create)
2. Create a workspace and invite yourself.
3. Open the Slack workspace in a browser (e.g. `https://yourworkspace.slack.com`)

---

## ✅ Step 2: Create a Slack App (Bot)

1. Go to: [https://api.slack.com/apps](https://api.slack.com/apps)
2. Click **Create New App** → **From scratch**
3. Name your app (e.g. `DemoBot`)
4. Choose your Slack workspace

---

## ✅ Step 3: Add Permissions (OAuth Scopes)

1. In the app dashboard, go to **OAuth & Permissions**
2. Under **Bot Token Scopes**, add:
   - `chat:write`
   - `channels:read` *(optional if using public channels)*
3. Click **Save Changes**

---

## ✅ Step 4: Install App to Workspace

1. Go to **Install App** tab
2. Click **Install to Workspace**
3. Click **Allow**
4. You’ll receive a **Bot User OAuth Token** like:

---

## ✅ Step 5: Invite Bot to Channel

Before your bot can post messages, it must be invited to the channel.

1. Open your Slack workspace in a browser or desktop app.
2. Navigate to the `#social` channel (or any channel you want the bot to message).
3. Type the following into the message input:
`/invite @DemoApp` or the other identifier you used fro your bot. After you type `/invite` there will be list of users and bots, slect yours
4. Press Enter. You should see a confirmation like:
`@DemoApp has joined the channel` Now the bot has permission to send messages to `#social`.
5. run two code cells below
6. After running python code below - look into chat you should see

```
Demo App APP  1:20 PM
Hello from my Slack bot demo! :wave:
```

In [ ]:
!pip install slack_sdk

In [ ]:
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

SLACK_BOT_TOKEN = "<your_slack_bot_token>"  # Starts with 'xoxb-'
CHANNEL_NAME = "#social"

client = WebClient(token=SLACK_BOT_TOKEN)

try:
    response = client.chat_postMessage(channel=CHANNEL_NAME, text="Hello from my Slack bot demo! 👋")
    print("✅ Message sent! Timestamp:", response["ts"])
except SlackApiError as e:
    print("❌ Error sending message:", e.response["error"])


# 🌍 GDELT + BigQuery Integration Guide

## ✅ What is GDELT?

The **Global Database of Events, Language, and Tone (GDELT)** is a massive open-source database that monitors news media from around the world in real time. It tracks:
- Geopolitical and local events
- Themes (health, drought, elections, conflict, etc.)
- Involved actors and locations
- Emotional tone and media framing

GDELT is publicly hosted in **Google BigQuery** and is freely accessible under the dataset:  
`gdelt-bq.gdeltv2.gkg`

---

## 🔐 Authentication (Google Colab or Notebook)

To use GDELT in a notebook or Colab:  
1. You will be prompted to `authenticate_user()` using your Google account  
2. A popup will appear asking for permissions  
3. You must allow access to continue — this happens once per session

Authentication enables the use of Google Cloud's BigQuery Python client for querying public datasets like GDELT.

---

## ⚡ Best Practices for GDELT Queries

- Always filter using `_PARTITIONTIME` or `DATE(_PARTITIONTIME)` to limit the query scope
- Always add a `LIMIT` to ensure fast performance
- Filter by `V2Themes` for topic-based exploration (e.g., `HLTH_DISEASE_OUTBREAK`, `ENV_DROUGHT`)
- Avoid `COUNT(*)` across wide date ranges — it’s slow and expensive

---

## ✅ Example Queries (GDELT GKG)

### Example 1: Quick Connectivity Check

Returns a single article URL from the past day:

```sql
SELECT DocumentIdentifier
FROM `gdelt-bq.gdeltv2.gkg`
WHERE _PARTITIONTIME = TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 1 DAY)
LIMIT 1
```

---

### Example 2: Disease Outbreak Mentions

Finds articles with the `HLTH_DISEASE_OUTBREAK` theme from yesterday:

```sql
SELECT V2Themes, DocumentIdentifier, V2Tone
FROM `gdelt-bq.gdeltv2.gkg`
WHERE _PARTITIONTIME = TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 1 DAY)
AND V2Themes LIKE '%HLTH_DISEASE_OUTBREAK%'
LIMIT 5
```

---

### Example 3: Drought-Themed News (Last 3 Days)

Fetches article links related to droughts in recent days:

```sql
SELECT DocumentIdentifier, V2Themes
FROM `gdelt-bq.gdeltv2.gkg`
WHERE DATE(_PARTITIONTIME) >= DATE_SUB(CURRENT_DATE(), INTERVAL 3 DAY)
AND V2Themes LIKE '%ENV_DROUGHT%'
LIMIT 10
```

---

### Example 4: Conflict Tone Analysis

Gets tone values for conflict-tagged news from yesterday:

```sql
SELECT V2Tone, DocumentIdentifier
FROM `gdelt-bq.gdeltv2.gkg`
WHERE _PARTITIONTIME = TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 1 DAY)
AND V2Themes LIKE '%WB_153_CONFLICT%'
LIMIT 5
```

---

## 📌 GDELT Fields to Know

- `DocumentIdentifier`: Original article URL
- `V2Themes`: Media-detected theme tags (comma-separated)
- `V2Locations`: Named location mentions
- `V2Tone`: Article sentiment (positive = optimistic, negative = pessimistic)


---

# ☁️ Google Cloud Setup for BigQuery Access (Free Tier)

Follow these steps to create a project, enable billing, and query GDELT using your own credentials.

---

## ✅ Step 1: Create Google Cloud Project

1. Visit: [https://console.cloud.google.com/projectcreate](https://console.cloud.google.com/projectcreate)
2. Enter a **Project Name** (e.g., `gdelt-demo-project`)
3. Click **Create**
4. After a few seconds, it will assign you a **Project ID** like:  
   ``gdelt-demo-project-397b4``  
   (This is what we will use as `your-google-cloud-project-id`)

---

## ✅ Step 2: Enable Billing (Free Tier)

> ☝️ You won't be charged unless you exceed free tier limits

1. Go to: [https://console.cloud.google.com/billing](https://console.cloud.google.com/billing)
2. Set up billing with a credit/debit card (required by Google)
3. Link your new project to the billing account

---

## ✅ Step 3: Enable BigQuery API

1. Go to: [https://console.cloud.google.com/apis/library/bigquery.googleapis.com](https://console.cloud.google.com/apis/library/bigquery.googleapis.com)
2. Click **Enable**

---

## ✅ Step 4: Run Demo Query in Colab

In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.cloud import bigquery

# this project `gdelt-bigquery` may limit quota and run too long!!!
#in Chapter 11 we cover GCP and billing, so you can create your own project and setup your BigData
client = bigquery.Client(project="gdelt-bigquery")



## connectivity test
query_job = client.query("SELECT 1")
results = query_job.result()
for row in results:
    print(row)


## real query
query = """
SELECT DocumentIdentifier, V2Themes
FROM `gdelt-bq.gdeltv2.gkg`
WHERE DATE(_PARTITIONTIME) >= DATE_SUB(CURRENT_DATE(), INTERVAL 3 DAY)
AND V2Themes LIKE '%ENV_DROUGHT%'
LIMIT 10
"""

query_job = client.query(query)

for row in query_job:
    print("✅ GDELT connection works. Example Document URL:\n", row.DocumentIdentifier)


# 🗞️ News APIQuick Demo

## Step 1: Get Free API Key

1. Sign up at [https://newsapi.org](https://newsapi.org)
2. After email verification, go to your account to copy the **API Key**

## Step 2: Run the Python Code Below




In [ ]:
import requests

# Replace with your actual NewsAPI key
NEWS_API_KEY = "<your_news_api_key>"  # Get one from https://newsapi.org/register
url = f"https://newsapi.org/v2/top-headlines?country=us&apiKey={NEWS_API_KEY}"

response = requests.get(url)
data = response.json()

for i, article in enumerate(data["articles"][:5], 1):
    print(f"\n{i}. {article['title']}")
    print(f"   {article['url']}")


